# Feb 15, 2025: membership histograms
for each community per mode marginals and soft-marginals

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source ='allen' #'spatial' #'allen'
args.space = 'ccfv2' #'yale' #'ccfv2'
args.brain_div = 'whl' #'whl' #'whl'
args.num_rois = 172 #162 #172
args.resolution = 200 #200 #200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
TS_path = f'{ROI_path}/roi_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
DIAG_path = f'{ROI_RESULTS_path}/diagnostics'
os.system(f'mkdir -p {DIAG_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
args.dc, args.sbm = False, 'h'

args.nested = True if args.sbm in ['h'] else False

args.force_niter = 60000 #40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['m', 'a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-nd-h'

In [6]:
def get_membership_matrix(num_rois, df, col='pi'):
    pis = [np.zeros((num_rois, 1)) if np.isnan(pi).all() else pi for pi in df[col]]

    num_modes = len(df)
    num_comms = np.max([pi.shape[-1] for pi in pis])
    num_rois = num_rois
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode, pi in enumerate(pis):
        M[:, idx_mode, :pi.shape[-1]] = pi
    
    return M

In [7]:
marginals_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-mem-mats.pkl', recursive=True))
marginals_df = []
for sbm_file in marginals_files:
    with open(f'{sbm_file}', 'rb') as f:
        row = pickle.load(f)
    marginals_df += [row]
marginals_df = pd.concat(marginals_df).reset_index(drop=True)
mode_ids = list(chain.from_iterable([list(range(count)) for count in marginals_df['sub'].value_counts().sort_index().to_list()]))
marginals_df['mode_id'] = mode_ids
marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned,omega,mode_id
0,SLC01,sbm-nd-h,"[[0.9961759082217974, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9281429025807136, 0.015237872401098237, 0....","[[0.9040823620086329, 0.09301573223162013, 0.0...","[[0.9874769137876359, 0.01252308621236418], [0...","[[0.9979963062060219, 0.0020036937939782685], ...","[[0.9994990765515056, 0.0005009234484945671], ...","[[1.0000000000000002], [1.0], [0.9999999999999...","[[1.0000000000000002], [1.0], [0.9999999999999...","[[1.0000000000000002], [1.0], [0.9999999999999...",0.523242,0
1,SLC01,sbm-nd-h,"[[0.7863777089783281, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9315904065253963, 0.015346972829780878, 0....","[[0.9145655675302087, 0.07874076010687803, 0.0...","[[0.9880659283123276, 0.011934071687672554], [...","[[0.9976131856624657, 0.002386814337534511], [...","[[1.0000000000000002], [1.0], [0.9999999999999...","[[1.0000000000000002], [1.0], [0.9999999999999...","[[1.0000000000000002], [1.0], [0.9999999999999...","[[1.0000000000000002], [1.0], [0.9999999999999...",0.322986,1
2,SLC01,sbm-nd-h,"[[0.9025974025974026, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9145724405464665, 0.006114015854275595, 0....","[[0.9005714514840302, 0.09785663622567302, 4.1...","[[0.9854256073706443, 0.014574392629355558], [...","[[0.9999999999999999], [1.0], [1.0000000000000...","[[0.9999999999999999], [1.0], [1.0000000000000...","[[0.9999999999999999], [1.0], [1.0000000000000...","[[0.9999999999999999], [1.0], [1.0000000000000...","[[0.9999999999999999], [1.0], [1.0000000000000...",0.153692,2
3,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.000080,3
4,SLC03,sbm-nd-h,"[[0.999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, ...","[[0.797203, 0.0, 0.0, 0.0, 0.0, 0.201798, 0.0,...","[[0.6391699612673797, 0.27201264538502673, 0.0...","[[0.9570630218886069, 0.0419938390955657, 0.00...","[[0.9902540201365801, 0.009046044904559113, 0....","[[0.998190791019088, 0.0018092089809118228], [...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.9999999999999999], [1.0], [0.9999999999999...",1.000000,0
5,SLC04,sbm-nd-h,"[[0.021778584392014518, 0.0, 0.0, 0.0, 0.0, 0....","[[0.1025910335175855, 0.009577789778479609, 0....","[[0.8480188772586088, 0.1465628387418961, 0.00...","[[0.9690042448867991, 0.030995755113200806], [...","[[0.9951569132635624, 0.004843086736437627], [...","[[0.9990313826527125, 0.0009686173472875254], ...","[[1.0], [1.0000000000000002], [1.0000000000000...","[[1.0], [1.0000000000000002], [1.0000000000000...","[[1.0], [1.0000000000000002], [1.0000000000000...",0.550712,0
6,SLC04,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.03823529411764706, 0.47941176470588237, 0....","[[0.8586792753640304, 0.12726534359979688, 0.0...","[[0.9722714932561584, 0.027728506743841676], [...","[[0.9933979745847996, 0.0066020254152003984], ...","[[0.9986795949169599, 0.0013204050830400798], ...","[[0.9986795949169599, 0.0013204050830400798], ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.339706,1
7,SLC04,sbm-nd-h,"[[0.4642857142857143, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.05044437129690587, 0.5307151086240948, 0.0...","[[0.8583275654702285, 0.11734920437365372, 0.0...","[[0.9679781393396721, 0.03202186066032775], [0...","[[0.9893260464465573, 0.010673953553442583], [...","[[0.9893260464465573, 0.010673953553442583], [...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.99

In [8]:
cols = [col for col in  list(marginals_df.columns) if 'pi_' in col]
cols

['pi_0_aligned',
 'pi_1_aligned',
 'pi_2_aligned',
 'pi_3_aligned',
 'pi_4_aligned',
 'pi_5_aligned',
 'pi_6_aligned',
 'pi_7_aligned',
 'pi_8_aligned']

In [9]:
# SOFT MARGINALS
soft_marginals_df = []
for sub, group in marginals_df.groupby('sub'):
    omegas = group['omega'].to_list()
    dct = {'sub': [sub], 'sbm': [SBM]}
    for col in cols:
        M = get_membership_matrix(args.num_rois, group, col=col)
        SCs = np.average(M, axis=1, weights=omegas) # soft-comms.
        dct[col] = [SCs]
    soft_marginals_df += [pd.DataFrame(dct)]
soft_marginals_df = pd.concat(soft_marginals_df).reset_index(drop=True)
soft_marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned
0,SLC01,sbm-nd-h,"[[0.9140320714701717, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9271764711025405, 0.01386962471182265, 0.0...","[[0.9069363630655839, 0.08914168617775663, 0.0...","[[0.9873528890732869, 0.012647110926713166], [...","[[0.9981806763026447, 0.0018193236973553643], ...","[[0.9997378958834185, 0.00026210411658168407],...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1...","[[1.0], [1.0], [0.9999999999999999], [1.0], [1..."
1,SLC03,sbm-nd-h,"[[0.999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, ...","[[0.797203, 0.0, 0.0, 0.0, 0.0, 0.201798, 0.0,...","[[0.6391699612673797, 0.27201264538502673, 0.0...","[[0.9570630218886069, 0.0419938390955657, 0.00...","[[0.9902540201365801, 0.009046044904559113, 0....","[[0.998190791019088, 0.0018092089809118228], [...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.9999999999999999], [1.0], [0.9999999999999..."
2,SLC04,sbm-nd-h,"[[0.03780388137638709, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.0782634830110574, 0.19763652598179973, 0.4...","[[0.8560124717624737, 0.13317940216894186, 0.0...","[[0.9708275493085543, 0.029172450691445705], [...","[[0.9944967323757501, 0.00550326762424986], [0...","[[0.9984246449733333, 0.0015753550266668075], ...","[[0.9995514509368364, 0.00044854906316359135],...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
3,SLC05,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.021994437874249345, 0.16699961117587434, 0...","[[0.8957240095070945, 0.09352806421293049, 3.4...","[[0.9841003187388588, 0.015899681261141246], [...","[[0.9980996092218923, 0.0019003907781077888], ...","[[1.0000000000000002], [1.0], [1.0000000000000...","[[1.0000000000000002], [1.0], [1.0000000000000...","[[1.0000000000000002], [1.0], [1.0000000000000...","[[1.0000000000000002], [1.0], [1.0000000000000..."
4,SLC06,sbm-nd-h,"[[0.003991848468513804, 0.0, 0.0, 0.0, 0.0, 0....","[[0.03605390205558599, 0.0, 0.0029722694152681...","[[0.9039735149563888, 0.07146064723315923, 7.6...","[[0.9835952500313778, 0.015842175264198812, 0....","[[0.9978254525594749, 0.0021745474405251717], ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
5,SLC07,sbm-nd-h,"[[0.0009995794766053622, 0.0, 0.0, 0.0, 0.0, 0...","[[2.4105614387589117e-05, 0.9549979224673795, ...","[[0.8562490896578213, 0.14068624946510788, 0.0...","[[0.9756797456617119, 0.0240767740687209, 0.0,...","[[0.9952309569996833, 0.0047690430003165885], ...","[[0.9993296929645852, 0.000670307035414724], [...","[[0.9999999999999998], [1.0000000000000002], [...","[[0.9999999999999998], [1.0000000000000002], [...","[[0.9999999999999998], [1.0000000000000002], [..."
6,SLC08,sbm-nd-h,"[[0.06598510095089652, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.0807792658994191, 0.07311335293063623, 0.3...","[[0.8911651285366838, 0.09446259624603848, 0.0...","[[0.9843144711187627, 0.015685528881237375], [...","[[0.9963729733046209, 0.0036270266953790874], ...","[[0.9996609531772318, 0.0003390468227682365], ...","[[0.9996609531772318, 0.0003390468227682365], ...","[[1.0], [1.0000000000000004], [1.0], [1.000000...","[[1.0], [1.0000000000000004], [1.0], [1.000000..."
7,SLC09,sbm-nd-h,"[[0.003994266212996924, 0.0, 0.0, 0.0, 0.0, 0....","[[0.07505715673603608, 0.00030235481293602217,...","[[0.9156563603243291, 0.06782596797786075, 0.0...","[[0.9873506729729487, 0.011984220965542622, 0....","[[0.9970944861672909, 0.0029055138327092086], ...","[[0.9993906067453178, 0.0006093932546822618], ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0000000...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0000000...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.000000

historgams 

In [10]:
def mem_bins_per_marginal(args, pi):
    if np.isnan(pi).all(): return np.nan
    # membweahip histogram per marginal
    bins = np.arange(0, 1+args.binwidth, args.binwidth)
    thresh = (bins[1] + bins[0]) / 2 # if bins are 0, 2.5, ..., then thresh = 0.25 / 2
    h = np.zeros((len(bins)-1, pi.shape[-1]))
    for col in range(pi.shape[-1]):
        vals = pi[:, col]
        vals = vals[vals > 0.1]
        hist, _ = np.histogram(vals, bins=bins)
        h[:, col] = hist
    h /= np.expand_dims(np.sum(h, axis=0), axis=0)
    h = np.round(np.nan_to_num(h), decimals=3)
    return h

In [11]:
args.binwidth = 0.15
bins_df = []
for idx, row in marginals_df.iterrows():
    sub = row['sub']
    sbm = row['sbm']
    mode_id = row['mode_id']
    omega = row['omega']
    
    dct = {'sub': [sub], 'sbm': [sbm], 'mode_id': [mode_id], 'omega': [omega]}
    for col in cols:
        pi = row[col]
        h = mem_bins_per_marginal(args, pi)
        dct[col] = [h]
    bins_df += [pd.DataFrame(dct)]
bins_df = pd.concat(bins_df).reset_index(drop=True)
bins_df

,sub,sbm,mode_id,omega,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned
0,SLC01,sbm-nd-h,0,0.523242,"[[0.0, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.0, 0.086, 0.415, 0.0, 0.0, 0.174, 0.052, 0...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
1,SLC01,sbm-nd-h,1,0.322986,"[[0.111, 0.0, 0.0, 0.0, 0.0, 0.2, 0.071, 0.0, ...","[[0.212, 0.135, 0.0, 0.0, 0.0, 0.0, 0.115, 0.0...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
2,SLC01,sbm-nd-h,2,0.153692,"[[0.2, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.125, ...","[[0.059, 0.19, 0.226, 0.0, 0.114, 0.042, 0.054...","[[0.0, 0.645, 0.0, 0.0, 0.0], [0.0, 0.355, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
3,SLC01,sbm-nd-h,3,0.000080,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
4,SLC03,sbm-nd-h,0,1.000000,"[[0.0, 0.0, 0.143, 0.0, 0.062, 0.0, 0.0, 0.111...","[[0.0, 0.5, 0.048, 0.0, 0.0, 0.378, 0.0, 0.0, ...","[[0.0, 0.128, 0.0, 0.0, 0.12, 0.0, 0.0, 0.0], ...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
5,SLC04,sbm-nd-h,0,0.550712,"[[0.0, 0.0, 0.0, 0.083, 0.0, 0.25, 0.0, 0.286,...","[[0.522, 0.033, 0.0, 0.086, 0.0, 0.056, 0.129,...","[[0.0, 0.697, 0.0, 0.0, 0.0], [0.0, 0.303, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
6,SLC04,sbm-nd-h,1,0.339706,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.222, 0....","[[0.371, 0.085, 0.034, 0.0, 0.0, 0.105, 0.048,...","[[0.0, 0.605, 0.0, 0.0, 0.0], [0.0, 0.395, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
7,SLC04,sbm-nd-h,2,0.055591,"[[0.0, 0.091, 0.0, 0.0, 0.0, 0.429, 0.0, 0.0, ...","[[0.241, 0.043, 0.222, 0.0, 0.238, 0.105, 0.09...","[[0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0

In [12]:
soft_bins_df = []
for idx, row in soft_marginals_df.iterrows():
    sub = row['sub']
    sbm = row['sbm']

    dct = {'sub':[sub], 'sbm':[sbm]}
    for col in cols:
        pi = row[col]
        h = mem_bins_per_marginal(args, pi)
        dct[col] = [h]
    soft_bins_df += [pd.DataFrame(dct)]
soft_bins_df = pd.concat(soft_bins_df).reset_index(drop=True)
soft_bins_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned
0,SLC01,sbm-nd-h,"[[0.0, 0.062, 0.125, 0.0, 0.0, 0.0, 0.143, 0.0...","[[0.03, 0.019, 0.03, 0.161, 0.071, 0.136, 0.07...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
1,SLC03,sbm-nd-h,"[[0.0, 0.0, 0.143, 0.0, 0.062, 0.0, 0.0, 0.111...","[[0.0, 0.5, 0.048, 0.0, 0.0, 0.378, 0.0, 0.0, ...","[[0.0, 0.128, 0.0, 0.0, 0.12, 0.0, 0.0, 0.0], ...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
2,SLC04,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.154, 0.0, 0.0, 0.0, 0.143, ...","[[0.208, 0.0, 0.0, 0.0, 0.0, 0.1, 0.091, 0.378...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
3,SLC05,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.182, 0.0, 0....","[[0.15, 0.345, 1.0, 0.37, 0.179, 0.034, 0.017,...","[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
4,SLC06,sbm-nd-h,"[[0.071, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0....","[[0.0, 0.13, 0.0, 0.034, 0.263, 0.107, 0.0, 0....","[[0.0, 0.275, 0.0, 0.0, 0.0, 0.0], [0.0, 0.725...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
5,SLC07,sbm-nd-h,"[[0.0, 0.071, 0.143, 0.067, 0.0, 0.2, 0.0, 0.0...","[[0.02, 0.024, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0],...","[[0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0....","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
6,SLC08,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.083, 0.0, 0.0, 1.0, 0.111, ...","[[0.225, 0.024, 0.027, 0.143, 0.2, 0.0, 0.029,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]"
7,SLC09,sbm-nd-h,"[[0.077, 0.0, 0.167, 0.0, 0.357, 0.0, 0.125, 0...","[[0.102, 0.109, 0.273, 0.0, 0.24, 0.0, 0.091, ...","[[0.0, 0.069, 0.0, 0.0, 0.0, 0.0], [0.0, 0.931...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1

plot histograms per animal

In [13]:
# plot membership histograms per mode, and overlay soft-marginal histograms 

In [14]:
bins = np.arange(0, 1+args.binwidth, args.binwidth)
args.num_bins = len(bins) - 1
xs = (bins[1:] + bins[:-1]) / 2

In [15]:
bins, xs

(array([0.  , 0.15, 0.3 , 0.45, 0.6 , 0.75, 0.9 , 1.05]),
 array([0.075, 0.225, 0.375, 0.525, 0.675, 0.825, 0.975]))

In [16]:
def plot_sub_hists_per_comm(idx_comm, ax, hs, soft_h):
    ax.plot(xs, hs[:, 0], marker='o', c='grey', markersize=7, linewidth=1, alpha=0.3, label='per mode')
    ax.plot(xs, hs[:, 1:], marker='o', c='grey', markersize=7, linewidth=1, alpha=0.3)
    ax.plot(xs, soft_h, marker='o', c='cornflowerblue', markersize=7, linewidth=3, alpha=1.0, label='across modes')
    ax.set(title=f'comm {idx_comm:02d}', xlabel=f'membership', ylabel=f'roi proportion')
    ax.grid(alpha=0.3)
    ax.legend()
    return ax

In [17]:
def plot_histograms_per_sub(args, sub, bdf, srow, col):
    H = get_membership_matrix(args.num_bins, bdf, col) 
    # using the same function to collect all matrices (membership/hist.bins)
    soft_h = srow[col].to_list()[0]
    
    ncols = 5
    nrows = np.ceil(H.shape[-1] / ncols).astype(int)
    fig, axs = plt.subplots(nrows, ncols, figsize=(5*ncols, 4*nrows))
    fig.tight_layout(h_pad=3, w_pad=3)

    level = col.split('_')[-1]
    sbm_title = f'{SBM} level-{level}' if level != 'aligned' else f'{SBM}'
    fig.suptitle(f'{sbm_title} {sub}', x=0.0, y=1.0)

    for idx_comm in range(H.shape[-1]):
        r, c = idx_comm // ncols, idx_comm % ncols
        ax = axs[r,c] if nrows > 1 else axs[c]
        ax = plot_sub_hists_per_comm(
            idx_comm, ax, 
            hs=H[:, :, idx_comm], 
            soft_h=soft_h[:, idx_comm]
        )
    for c_ in range(c+1, ncols):
        ax = axs[r, c_] if nrows > 1 else axs[c_]
        fig.delaxes(ax)
    return fig 

In [18]:
groups = list(bins_df.groupby('sub'))

for col in cols:
    for sub, bdf in tqdm(groups):
        srow = soft_bins_df[soft_bins_df['sub'] == sub]
        fig = plot_histograms_per_sub(args, sub, bdf, srow, col)

        folder = f'{ESTIM_path}/individual/sub-{sub}/membership-histograms/{SBM}'
        os.system(f'mkdir -p {folder}')
        fig.savefig(f'{folder}/desc-{col}.pdf', bbox_inches='tight')
        # break
    plt.close('all')

100%|██████████| 9/9 [00:02<00:00,  4.15it/s]


plot histograms across animals

In [19]:
# plot soft-marginal histograms per animal and their bootstrap mean

In [20]:
def bootstrap_histogram_means(matrix, n_bootstrap=1000, confidence_level=0.95):
    n_realizations, n_bins = matrix.shape
    
    compute_sample_mean = lambda x: np.mean(x, axis=0)
    
    # bootstrap
    bootstrap_means = np.array([
        compute_sample_mean(matrix[np.random.choice(n_realizations, size=n_realizations, replace=True), :])
        for _ in range(n_bootstrap)
    ])
    
    # statistics
    mean_estimate = np.mean(bootstrap_means, axis=0)
    
    # confidence intervals
    # in case of bootstrapping, one can calculate CI as follows:
    # sort all bootstrap means in ascending order 
    # lower bound = confidence_level / 2 th percentile of the bootstrap means
    # upper bound = 100 - confidence_level / 2 th percentile of the bootstrap means
    ci_lower = np.percentile(bootstrap_means, q=100*(1 - confidence_level) / 2, axis=0)
    ci_upper = np.percentile(bootstrap_means, q=100 * (1 - (1 - confidence_level) / 2), axis=0)
    # 95% CI based on standard error of the mean
    # vs
    # 95% CI based on standard deviation of data
    
    return mean_estimate, ci_lower, ci_upper

In [21]:
def plot_soft_hists_per_comm(idx_comm, ax, xs, hs, mu, cil, ciu):
    ax.plot(xs, hs[:, 0], marker='o', c='grey', markersize=7, fillstyle='none', linewidth=1, alpha=0.3, label='animal')
    ax.plot(xs, hs[:, 1:], marker='o', c='grey', markersize=7, fillstyle='none', linewidth=1, alpha=0.3)

    ax.plot(xs, mu, marker='o', c='cornflowerblue', markersize=10, linewidth=3, alpha=1.0, label='mean')
    ax.fill_between(
        x=xs, 
        y1=cil,
        y2=ciu,
        color='cornflowerblue',
        alpha=0.3,
        label='95% CI'
    )
    ax.legend()
    ax.grid(alpha=0.3)
    ax.set(title=f'comm {idx_comm:02d}', xlabel=f'membership', ylabel=f'roi proportion')
    return ax

In [22]:
def plot_soft_hists(H):
    ncols = 5
    nrows = np.ceil(H.shape[-1] / ncols).astype(int)
    fig, axs = plt.subplots(nrows, ncols, figsize=(5*ncols, 4*nrows))
    fig.tight_layout(h_pad=3, w_pad=3)

    level = col.split('_')[-1]
    sbm_title = f'{SBM} level-{level}' if level != 'aligned' else f'{SBM}'
    fig.suptitle(f'{sbm_title}', x=0.0, y=1.0)

    for idx_comm in range(H.shape[-1]):
        r, c = idx_comm // ncols, idx_comm % ncols
        ax = axs[r, c] if nrows > 1 else axs[c]

        hs = H[:, :, idx_comm]
        (mu, cil, ciu) = bootstrap_histogram_means(H[:, :, idx_comm].T)
        ax = plot_soft_hists_per_comm(idx_comm, ax, xs, hs, mu, cil, ciu)

    for c_ in range(c+1, ncols):
        ax = axs[r, c_] if nrows > 1 else axs[c_]
        fig.delaxes(ax)
    return fig  

In [23]:
for col in tqdm(cols):
    H = get_membership_matrix(args.num_bins, soft_bins_df, col=col)
    fig = plot_soft_hists(H)

    folder = f'{ESTIM_path}/group/membership-histograms/{SBM}'
    os.system(f'mkdir -p {folder}')
    fig.savefig(f'{folder}/desc-{col}.pdf', bbox_inches='tight')
plt.close('all')

100%|██████████| 9/9 [00:15<00:00,  1.76s/it]
